# CNN 개요

![](https://d.pr/i/nSRcmK+)

![](https://d.pr/i/yCJQyk+)

# Convolution Layer 주요 개념

### Feature Extraction

### Filter (Kernel)
![https://user-images.githubusercontent.com/15958325/58780750-defb7480-8614-11e9-943c-4d44a9d1efc4.gif](https://user-images.githubusercontent.com/15958325/58780750-defb7480-8614-11e9-943c-4d44a9d1efc4.gif)

![https://miro.medium.com/0*zADmyHH92LZPdnmj.gif](https://miro.medium.com/0*zADmyHH92LZPdnmj.gif)

### Stride

![https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1
](https://miro.medium.com/v2/resize:fit:588/format:webp/1*BMngs93_rm2_BpJFH2mS0Q.gif)

### Padding

![https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1
](https://miro.medium.com/v2/resize:fit:640/format:webp/1*1okwhewf5KCtIPaFib4XaA.gif)

### Pooling(Sub-sampling)

![https://towardsdatascience.com/convolutional-neural-network-1368ee2998d3](https://d.pr/i/5S0uPJ+)


![http://ufldl.stanford.edu/tutorial/supervised/Pooling/
](https://d.pr/i/I9XtBk+)


### Channel & Filter

![https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*8dx6nxpUh2JqvYWPadTwMQ.gif)

![https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1
](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*CYB2dyR3EhFs1xNLK8ewiA.gif)


![https://cs231n.github.io/convolutional-networks/](https://d.pr/i/RMxib5+)

### Conv + Pool

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

input_tensor = torch.randn(1, 1, 28, 28) # (batch_size, channel, height, width) → 1x1x28x28 크기의 텐서 → 1개의 채널, 28x28 크기의 이미지(흑백)

conv = nn.Conv2d(
    in_channels=1,           # 입력 채널 수/필터수
    out_channels=4,          # 출력 채널 수 (현재 레이어의 필터 수) → 4개의 필터가 1개의 batch에 적용됨, 1개의 채널에 4개의 필터가 적용됨 → 4개의 채널이 출력됨 = 4개의 feature map
    kernel_size=3,           # 필터(커널) 크기 (3, 3) → 3x3 크기의 필터
    stride=1,                # 필터 이동 크기
    padding='same'           # 기본값: valid(패딩 사용 안함), 'same' → 입력 크기와 동일한 크기로 패딩 사용, 출력 결과도 동일한 크기
)

pool = nn.MaxPool2d(         # kernel_size 와 stride를 보통 같게 설정함   
    kernel_size=2,           # 풀링 크기 2x2
    stride=2                 # 스트라이드 2
)  

output = F.relu(conv(input_tensor))  # 입력 텐서에 필터 적용 → conv + relu 적용
print(output.shape)          # 출력 텐서의 크기 확인 → torch.Size([1, 4, 28, 28])
                             # padding='valid'의 경우 출력 크기가 작아짐 → torch.Size([1, 4, 26, 26]) → 4장의 Feature Map이 출력됨
output = pool(output)        # 풀링 적용 → Sub-Sampling, Pooling
print(output.shape)          # 출력 텐서의 크기 확인 → torch.Size([1, 4, 14, 14]) → 4장의 Feature Map이 14x14 크기로 풀링됨

torch.Size([1, 4, 28, 28])
torch.Size([1, 4, 14, 14])


In [6]:
# 필터 형태 확인
print(conv.weight.shape)     # 필터 형태 확인 → torch.Size([4, 1, 3, 3]) → 4개의 필터, 1개의 채널, 3x3 크기의 필터

torch.Size([4, 1, 3, 3])


In [8]:
conv.weight                  # 필터 형태 확인 → 4개의 필터(차원), 1개의 채널, 3x3 크기의 필터

Parameter containing:
tensor([[[[ 0.3206,  0.2786,  0.2502],
          [ 0.2850, -0.2942,  0.2875],
          [-0.3270,  0.0613,  0.2076]]],


        [[[ 0.0505, -0.1999,  0.2551],
          [ 0.1770,  0.0731, -0.0698],
          [-0.1260,  0.1657, -0.1710]]],


        [[[-0.1101, -0.1002, -0.2505],
          [ 0.3060, -0.0952, -0.0429],
          [ 0.0628, -0.1446,  0.2213]]],


        [[[-0.0930,  0.1399, -0.3250],
          [ 0.2533,  0.1161, -0.3112],
          [-0.1577, -0.2587, -0.0219]]]], requires_grad=True)

In [9]:
# 필터 형태 확인
print(conv.weight.shape)  # 4장의 필터
print(conv.bias.shape)    # 각 필터의 절편

torch.Size([4, 1, 3, 3])
torch.Size([4])


### CNN Model

1. Feature Extraction Layer
2. Fully Connected (Classification) Layer

In [17]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()                                            # 부모 클래스의 생성자 호출
        # 특성 추출 레이어
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding='same')      # 1개의 채널, 32개의 커널(필터), 3x3 크기의 커널
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding='valid')    # 32개의 채널, 64개의 커널(필터), 3x3 크기의 커널

        # 풀링 레이어
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # 완전 연결 (분류) 레이어
        self.flatten = nn.Flatten()                                                 # 2차원 텐서를 1차원 텐서로 변환
        self.fc1 = nn.Linear(64 * 13 * 13, 100)                                    # 64개의 채널, 13x13 크기의 특성 맵, 100개의 노드
        self.fc2 = nn.Linear(100, 10)                                               # 10개의 클래스(0~9)에 대한 출력 → 10개의 출력 → 10개의 클래스에 대한 출력, 10개 예측
    
    def forward(self, x):
        # 특성 추출 레이어
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        # 완전 연결 (분류) 레이어
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [18]:
model = CNNModel()
model

CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=10816, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

### CNN 모델 구조 상세 설명

#### 1. 왜 `nn.Linear(64 * 13 * 13, 100)`을 사용했나요?

##### 📐 입력 크기 추적 과정

입력 이미지가 각 레이어를 통과하면서 크기가 어떻게 변하는지 단계별로 추적해봅시다:

###### **초기 입력**: `(1, 1, 28, 28)`
- Batch: 1
- Channel: 1 (흑백 이미지)
- Height: 28
- Width: 28

---

###### **Step 1: Conv1 레이어 통과**
```python
self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding='same')
```
- **입력**: `(1, 1, 28, 28)`
- **출력**: `(1, 32, 28, 28)`
- `padding='same'` → 입력과 출력 크기 동일 유지
- 32개의 필터 → 32개 채널 생성

---

###### **Step 2: Conv2 레이어 통과**
```python
self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding='valid')
```
- **입력**: `(1, 32, 28, 28)`
- **출력**: `(1, 64, 26, 26)`
- `padding='valid'` → 패딩 없음
- **크기 계산**: `28 - 3 + 1 = 26` (양쪽에서 1픽셀씩 손실)
- 64개의 필터 → 64개 채널 생성

---

###### **Step 3: MaxPool 레이어 통과**
```python
self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
```
- **입력**: `(1, 64, 26, 26)`
- **출력**: `(1, 64, 13, 13)`
- 2×2 풀링 → 크기가 절반으로 감소
- **크기 계산**: `26 / 2 = 13`

---

###### **Step 4: Flatten 레이어 통과**
```python
self.flatten = nn.Flatten()
```
- **입력**: `(1, 64, 13, 13)`
- **출력**: `(1, 10816)`
- **계산**: `64 × 13 × 13 = 10,816`
- 3차원 텐서를 1차원으로 펼침

---

###### **Step 5: FC1 레이어 (첫 번째 완전 연결층)**
```python
self.fc1 = nn.Linear(64 * 13 * 13, 100)
```
- **입력 특성 개수**: `64 × 13 × 13 = 10,816`
- **출력 특성 개수**: `100`
- **의미**: 10,816개의 특성을 100개의 노드로 압축
- 이 레이어가 **특성 추출 → 분류 전환점**

---

#### 2. CNNModel() 출력 결과의 의미

```
CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=10816, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)
```

##### 🔍 각 레이어 상세 분석

| 레이어 | 파라미터 | 설명 |
|--------|----------|------|
| **conv1** | `1 → 32` | 입력 1채널에서 32개의 특성 맵 생성 |
| | `kernel_size=(3, 3)` | 3×3 크기의 필터 사용 |
| | `padding=same` | 출력 크기 = 입력 크기 유지 |
| **conv2** | `32 → 64` | 32채널에서 64개의 특성 맵 생성 |
| | `padding=valid` | 패딩 없음 (크기 감소) |
| **pool** | `kernel_size=2` | 2×2 영역에서 최댓값 추출 |
| | `stride=2` | 2칸씩 이동 (크기 절반) |
| **flatten** | `start_dim=1` | Batch 차원 제외하고 펼침 |
| **fc1** | `10816 → 100` | 특성 벡터를 100개 노드로 축소 |
| | `bias=True` | 편향(절편) 포함 |
| **fc2** | `100 → 10` | 10개 클래스 분류 (0~9 숫자) |

---

#### 3. 모델 구조 요약

##### 🎯 **2단계 구조**

###### **1단계: Feature Extraction (특성 추출)**
```
Input(28×28) → Conv1(same) → Conv2(valid) → Pool → Flatten
    ↓            ↓              ↓            ↓        ↓
  1×28×28    32×28×28      64×26×26    64×13×13  10,816
```

###### **2단계: Classification (분류)**
```
Flatten → FC1 → ReLU → FC2 → Output
   ↓       ↓              ↓       ↓
 10,816   100           100      10
```

---

#### 4. 파라미터 개수 계산

##### **Conv1 레이어**
- 가중치: `1 × 32 × 3 × 3 = 288`
- 편향: `32`
- **총**: `320`개

##### **Conv2 레이어**
- 가중치: `32 × 64 × 3 × 3 = 18,432`
- 편향: `64`
- **총**: `18,496`개

##### **FC1 레이어**
- 가중치: `10,816 × 100 = 1,081,600`
- 편향: `100`
- **총**: `1,081,700`개

##### **FC2 레이어**
- 가중치: `100 × 10 = 1,000`
- 편향: `10`
- **총**: `1,010`개

##### **전체 파라미터**: 약 **1,101,526개**

---

#### 5. 핵심 포인트 ⭐

1. **Flatten 이후 크기가 `64 × 13 × 13 = 10,816`이므로, FC1의 입력은 반드시 10,816이어야 합니다.**

2. **100은 설계자가 선택한 은닉층 노드 개수**입니다. 이 값은 다음 요인에 따라 달라집니다:
   - 모델의 표현력
   - 과적합 방지
   - 계산 효율성

3. **모델 출력은 모델의 "설계도"**를 보여줍니다:
   - 각 레이어의 타입
   - 입출력 차원
   - 하이퍼파라미터 설정

4. **데이터 흐름**: `이미지 → 특성 맵 → 벡터 → 분류 확률`

In [19]:
!pip install torchsummary

In [20]:
from torchsummary import summary
# model = model.cuda() # GPU 사용 → GPU 사용시 설정
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 26, 26]          18,496
         MaxPool2d-3           [-1, 64, 13, 13]               0
           Flatten-4                [-1, 10816]               0
            Linear-5                  [-1, 100]       1,081,700
            Linear-6                   [-1, 10]           1,010
Total params: 1,101,526
Trainable params: 1,101,526
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.69
Params size (MB): 4.20
Estimated Total Size (MB): 4.89
----------------------------------------------------------------


### CNN 모델 Summary 상세 분석

#### 🧠 Chain of Thought: 결과 분석 과정

각 값이 어떻게 계산되는지 단계별로 추적하여 깊이 있는 이해를 돕겠습니다.

---

#### 📊 레이어별 상세 분석

##### **Layer 1: Conv2d-1**

```
Layer (type)               Output Shape         Param #
Conv2d-1                  [-1, 32, 28, 28]         320
```

###### 🔍 Output Shape 분석: `[-1, 32, 28, 28]`

**CoT 과정:**
1. **입력**: `(Batch, 1, 28, 28)` → 흑백 이미지
2. **Conv2d(1, 32, kernel_size=3, padding='same')**
   - `padding='same'` → 출력 크기 = 입력 크기
   - 필터 32개 → 채널 32개 생성
3. **결과**: `(Batch, 32, 28, 28)`

**`-1`의 의미**: Batch 크기 (동적으로 변할 수 있음)
- 학습 시: 32, 64, 128 등
- 추론 시: 1, 10 등 가변적

##### 💡 Param # 계산: **320**

**공식**: `(입력 채널 × 출력 채널 × 커널 높이 × 커널 너비) + 출력 채널`

**단계별 계산:**
```
가중치(Weight) = 1 × 32 × 3 × 3 = 288개
편향(Bias)     = 32개 (각 필터마다 1개)
─────────────────────────────────────
총 파라미터    = 288 + 32 = 320개
```

**의미**: 
- 각 필터는 `1 × 3 × 3 = 9`개의 가중치를 가짐
- 32개 필터 → `9 × 32 = 288`
- 각 필터마다 절편 1개 추가

---

##### **Layer 2: Conv2d-2**

```
Layer (type)               Output Shape         Param #
Conv2d-2                  [-1, 64, 26, 26]       18,496
```

###### 🔍 Output Shape 분석: `[-1, 64, 26, 26]`

**CoT 과정:**
1. **입력**: `(Batch, 32, 28, 28)`
2. **Conv2d(32, 64, kernel_size=3, padding='valid')**
   - `padding='valid'` → 패딩 없음
   - **크기 계산 공식**: `(입력 - 커널 + 1) / stride`
   - Height: `(28 - 3 + 1) / 1 = 26`
   - Width: `(28 - 3 + 1) / 1 = 26`
   - 필터 64개 → 채널 64개
3. **결과**: `(Batch, 64, 26, 26)`

**왜 26인가?**
- 3×3 필터가 28×28 이미지를 스캔할 때
- 패딩이 없으면 가장자리에서 필터가 벗어남
- 좌우 각 1픽셀씩 손실 → `28 - 2 = 26`

###### 💡 Param # 계산: **18,496**

**단계별 계산:**
```
가중치(Weight) = 32 × 64 × 3 × 3 = 18,432개
               ↑    ↑    ↑   ↑
            입력  출력  커널  커널
            채널  채널  높이  너비

편향(Bias)     = 64개 (각 필터마다 1개)
─────────────────────────────────────
총 파라미터    = 18,432 + 64 = 18,496개
```

**중요 포인트**: 
- 입력 채널이 32개라는 것은 각 필터가 32개의 층을 가진다는 의미
- 즉, 각 필터의 실제 크기는 `32 × 3 × 3`

---

##### **Layer 3: MaxPool2d-3**

```
Layer (type)               Output Shape         Param #
MaxPool2d-3               [-1, 64, 13, 13]           0
```

###### 🔍 Output Shape 분석: `[-1, 64, 13, 13]`

**CoT 과정:**
1. **입력**: `(Batch, 64, 26, 26)`
2. **MaxPool2d(kernel_size=2, stride=2)**
   - 2×2 영역에서 최댓값만 추출
   - **크기 계산**: `26 / 2 = 13`
3. **결과**: `(Batch, 64, 13, 13)`

**시각화:**
```
[1 2]  →  [4]    (4개 → 1개로 압축)
[3 4]

26×26  →  13×13  (크기 1/4로 축소)
```

###### 💡 Param # 계산: **0**

**이유**: 
- 풀링은 **계산만 수행**, 학습 가능한 파라미터 없음
- 고정된 규칙(최댓값 선택)만 적용
- 가중치도, 편향도 필요 없음

---

##### **Layer 4: Flatten-4**

```
Layer (type)               Output Shape         Param #
Flatten-4                     [-1, 10816]            0
```

###### 🔍 Output Shape 분석: `[-1, 10816]`

**CoT 과정:**
1. **입력**: `(Batch, 64, 13, 13)` → 3차원 텐서
2. **Flatten**: 배치를 제외한 모든 차원을 1차원으로 펼침
3. **계산**: `64 × 13 × 13 = 10,816`
4. **결과**: `(Batch, 10816)` → 2차원 텐서

**시각화:**
```
입력:  [[[픽셀들...], 
         [픽셀들...], ...],  ← 64개 레이어
        [[픽셀들...], 
         [픽셀들...], ...]]

출력:  [픽셀1, 픽셀2, ..., 픽셀10816]  ← 일렬로 나열
```

###### 💡 Param # 계산: **0**

**이유**: 
- 단순히 데이터 **재배열**만 수행
- 어떤 연산이나 변환도 하지 않음
- 학습 가능한 파라미터 불필요

---

##### **Layer 5: Linear-5 (FC1)**

```
Layer (type)               Output Shape         Param #
Linear-5                      [-1, 100]        1,081,700
```

###### 🔍 Output Shape 분석: `[-1, 100]`

**CoT 과정:**
1. **입력**: `(Batch, 10816)`
2. **Linear(10816, 100)**: 완전 연결층
   - 10,816개 입력 뉴런
   - 100개 출력 뉴런
   - 모든 입력이 모든 출력에 연결
3. **결과**: `(Batch, 100)`

**의미**: 
- 고차원 특성(10,816) → 저차원 특성(100)으로 압축
- 중요한 특성만 추출

###### 💡 Param # 계산: **1,081,700** (모델에서 가장 많음!)

**단계별 계산:**
```
가중치(Weight) = 10,816 × 100 = 1,081,600개
                   ↑       ↑
                 입력    출력
                 노드    노드

편향(Bias)     = 100개 (각 출력 노드마다 1개)
─────────────────────────────────────────────
총 파라미터    = 1,081,600 + 100 = 1,081,700개
```

**왜 이렇게 많은가?**
- **완전 연결(Fully Connected)** → 모든 입력이 모든 출력에 연결
- 연결 개수 = 입력 × 출력 = `10,816 × 100`
- 각 연결마다 가중치 1개씩 필요

**시각화:**
```
입력(10,816개)          출력(100개)
    ●  ─────────────┐
    ●  ─────────────┼──→ ●
    ●  ─────────────┤    ●
   ...  ─────────────┤    ●
    ●  ─────────────┘   ...
                         ●
```

---

##### **Layer 6: Linear-6 (FC2)**

```
Layer (type)               Output Shape         Param #
Linear-6                       [-1, 10]          1,010
```

###### 🔍 Output Shape 분석: `[-1, 10]`

**CoT 과정:**
1. **입력**: `(Batch, 100)`
2. **Linear(100, 10)**: 최종 분류층
   - 100개 입력
   - 10개 출력 (클래스 0~9)
3. **결과**: `(Batch, 10)`

**의미**: 
- 각 클래스에 대한 점수(logit) 산출
- Softmax 적용 시 확률로 변환됨

###### 💡 Param # 계산: **1,010**

**단계별 계산:**
```
가중치(Weight) = 100 × 10 = 1,000개
편향(Bias)     = 10개
─────────────────────────────────────
총 파라미터    = 1,000 + 10 = 1,010개
```

---

#### 📈 전체 통계 분석

##### **Total params: 1,101,526**

**계산 검증:**
```
Conv2d-1:         320
Conv2d-2:      18,496
MaxPool2d-3:       0
Flatten-4:         0
Linear-5:  1,081,700
Linear-6:      1,010
─────────────────────
Total:     1,101,526 ✓
```

##### **Trainable params: 1,101,526**

**의미**: 
- 모든 파라미터가 학습 가능
- 역전파(Backpropagation)로 업데이트됨
- BatchNorm이나 고정된 층이 없음

##### **Non-trainable params: 0**

**Non-trainable이 생기는 경우:**
- BatchNormalization의 통계값 (mean, variance)
- 사전 학습된 층을 고정(freeze)했을 때
- Dropout (파라미터 자체가 없음)

---

#### 💾 메모리 사용량 분석

##### **Input size (MB): 0.00**

**계산 근거:**
```
입력 크기 = 1 (batch) × 1 (channel) × 28 × 28
         = 784개 값

Float32 기준: 784 × 4 bytes = 3,136 bytes
            = 0.003 MB ≈ 0.00 MB
```

##### **Forward/backward pass size (MB): 0.69**

**의미**: 
- Forward pass: 입력 → 출력 계산 시 중간 결과 저장
- Backward pass: 기울기(gradient) 계산 시 필요한 메모리

**왜 필요한가?**
- 역전파를 위해 각 레이어의 출력값을 저장해야 함
- Batch 크기가 커질수록 이 값도 증가

**대략적 계산:**
```
Conv1 출력:  32 × 28 × 28 = 25,088
Conv2 출력:  64 × 26 × 26 = 43,264
Pool 출력:   64 × 13 × 13 = 10,816
Flatten:     10,816
FC1:         100
FC2:         10
───────────────────────────────────
합계:        약 79,094 × 4 bytes (Float32)
            ≈ 0.3 MB (forward만)

Forward + Backward: 약 0.69 MB
```

##### **Params size (MB): 4.20**

**계산:**
```
총 파라미터:   1,101,526개
Float32 기준: 1,101,526 × 4 bytes
            = 4,406,104 bytes
            = 4.20 MB ✓
```

**의미**: 
- 모델 가중치 저장에 필요한 공간
- 모델 파일(.pth) 크기와 유사
- Optimizer 사용 시 2~3배 더 필요 (Adam은 파라미터당 2개 추가 저장)

##### **Estimated Total Size (MB): 4.89**

**계산:**
```
Input:               0.00 MB
Forward/backward:    0.69 MB
Params:              4.20 MB
───────────────────────────
Total:               4.89 MB ✓
```

**실제 학습 시 메모리:**
```
모델 파라미터:      4.20 MB
Optimizer 상태:     8.40 MB (Adam의 경우, 파라미터 × 2)
Forward 활성화:     0.69 MB × batch_size
Gradient:           4.20 MB
──────────────────────────────────
최소:               약 17.5 MB + (0.69 × batch_size) MB
```

---

#### 🎯 핵심 인사이트

##### 1. **파라미터 분포**
```
FC1 (1,081,700) ━━━━━━━━━━━━━━━━━━ 98.2%
Conv2 (18,496)  ━━ 1.7%
FC2 (1,010)     ▏ 0.09%
Conv1 (320)     ▏ 0.03%
```
→ **완전 연결층(FC1)이 압도적으로 많은 파라미터를 차지!**

### 2. **최적화 포인트**

**문제점**: FC1이 파라미터의 98%를 차지
- 과적합 위험 높음
- 계산 비용 큼
- 메모리 사용량 큼

**개선 방법:**
- **Global Average Pooling** 사용 → FC1 제거 가능
- **더 많은 Conv 층** 추가 → Pooling을 통해 크기 축소
- **Dropout** 추가 → 과적합 방지
- **L2 Regularization** 적용

##### 3. **Convolutional Layer의 효율성**

```
Conv1: 320개 파라미터로 28×28×32 = 25,088개 출력 생성
     → 파라미터 1개당 78개 출력값 생성 (효율적!)

FC1:   1,081,700개 파라미터로 100개 출력만 생성
     → 파라미터 1개당 0.0001개 출력 (비효율적)
```

##### 4. **메모리 최적화 전략**

**학습 시 메모리 절약 방법:**
- **Batch size 감소**: Forward/backward 메모리 감소
- **Mixed Precision (FP16)**: 메모리 1/2로 감소
- **Gradient Checkpointing**: Forward 중간값 재계산

---

#### 📚 학습 정리

##### ✅ 반드시 기억할 것

1. **파라미터 계산 공식**
   - Conv: `(in_ch × out_ch × k_h × k_w) + out_ch`
   - Linear: `(in_feat × out_feat) + out_feat`

2. **Output Shape 변화**
   - Conv (same): 크기 유지
   - Conv (valid): `(input - kernel + 1)`
   - Pooling: `input / stride`
   - Flatten: `channel × height × width`

3. **메모리 = 파라미터 + 활성화 + 기울기**

4. **FC층이 파라미터의 대부분을 차지** → 최적화 대상!